In [1]:
import pandas as pd
import numpy as np
import pickle
import csv

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option("future.no_silent_downcasting", True)

In [3]:
# https://ec.europa.eu/eurostat/databrowser/view/EF_LUS_MAIN/default/table?lang=en&category=agr.ef.ef_landuse  for agriculture
# https://ec.europa.eu/eurostat/databrowser/view/fish_fleet_alt/default/table?lang=en for fishing
# forestry data mimic agricultural ones as the number of forestry firms for EU is not available: eurostat suggests that the two sectors often overlap

In [4]:
eu27_list = ["Austria", "Belgium", "Bulgaria", "Croatia",
             "Cyprus", "Czechia", "Denmark", "Estonia",
            "Finland", "France", "Germany", "Greece",
            "Hungary", "Ireland", "Italy", "Latvia",
            "Lithuania", "Luxembourg", "Malta", "Netherlands",
            "Poland", "Portugal", "Romania", "Slovakia",
            "Slovenia", "Spain", "Sweden"
            ]

In [5]:
fish_eu22 = pd.read_excel("fish_eu22.xlsx")
fish_eu22 = fish_eu22.iloc[11:33, 0:2]
fish_eu22 = fish_eu22.reset_index(drop=True).rename(index=lambda x: x + 1)
fish_eu22.columns = ['Country', 'Fishing firms']

fish_eu27 = pd.DataFrame({'Country': eu27_list})
fish_eu27 = fish_eu27.merge(fish_eu22, on='Country', how='left')

In [6]:
agri_eu27 = pd.read_excel("agri_eu27.xlsx")
agri_eu27 = agri_eu27.iloc[12:39, 0:2]
agri_eu27 = agri_eu27.reset_index(drop=True).rename(index=lambda x: x + 1)
agri_eu27.columns = ['Country', 'Agricultural firms']
agri_eu27 = agri_eu27.set_index('Country').loc[eu27_list].reset_index()
agri_eu27['Fishing firms'] = fish_eu27['Fishing firms']


agri_eu27_shares = pd.DataFrame({'Country': eu27_list})
agri_eu27_shares['Agricultural share'] = agri_eu27['Agricultural firms'] / agri_eu27['Agricultural firms'].sum()
agri_eu27_shares['Forestry share'] = agri_eu27['Agricultural firms'] / agri_eu27['Agricultural firms'].sum()
agri_eu27_shares['Fishing share'] = agri_eu27['Fishing firms'] / agri_eu27['Fishing firms'].sum()
agri_eu27_shares = agri_eu27_shares.fillna(0)


agri_eu27_cum_shares = pd.DataFrame({'Country': eu27_list})
agri_eu27_cum_shares['Agricultural cumulative share'] = agri_eu27_shares['Agricultural share'].cumsum()
agri_eu27_cum_shares['Forestry cumulative share'] = agri_eu27_shares['Forestry share'].cumsum()
agri_eu27_cum_shares['Fishing cumulative share'] = agri_eu27_shares['Fishing share'].cumsum()

    
agri_eu27_cum_shares = agri_eu27_cum_shares.T
agri_eu27_cum_shares.columns = agri_eu27_cum_shares.iloc[0]
agri_eu27_cum_shares = agri_eu27_cum_shares[1:]
agri_eu27_cum_shares

Country,Austria,Belgium,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,Finland,France,Germany,Greece,Hungary,Ireland,Italy,Latvia,Lithuania,Luxembourg,Malta,Netherlands,Poland,Portugal,Romania,Slovakia,Slovenia,Spain,Sweden
Agricultural cumulative share,0.012213,0.016181,0.030815,0.046682,0.050436,0.053623,0.057712,0.058965,0.063995,0.107324,0.136293,0.194804,0.220386,0.234742,0.359648,0.367252,0.381813,0.38202,0.382864,0.388667,0.532238,0.564233,0.882509,0.884673,0.892662,0.993519,1.0
Forestry cumulative share,0.012213,0.016181,0.030815,0.046682,0.050436,0.053623,0.057712,0.058965,0.063995,0.107324,0.136293,0.194804,0.220386,0.234742,0.359648,0.367252,0.381813,0.38202,0.382864,0.388667,0.532238,0.564233,0.882509,0.884673,0.892662,0.993519,1.0
Fishing cumulative share,0,0.000842,0.025252,0.126086,0.136861,0.136861,0.163984,0.188421,0.230356,0.313197,0.330468,0.525894,0.525894,0.553084,0.715531,0.724461,0.726332,0.726332,0.738136,0.749285,0.760273,0.863367,0.865706,0.865706,0.867524,0.98567,1.0


In [7]:
agri_eu27_cum_shares.to_pickle("agri_eu27.xp")

In [8]:
# Share of people living in households owning or renting their home, 2023
# owner share: https://ec.europa.eu/eurostat/databrowser/view/ilc_lvho02__custom_12697765/bookmark/table?lang=en&bookmarkId=4b4bd471-585b-40a2-aabc-495ff4e56790&c=1724850706258
# avg household size: https://ec.europa.eu/eurostat/databrowser/view/ilc_lvph01/default/table?lang=en&utm_source=chatgpt.com
# number of households per country: https://ec.europa.eu/eurostat/databrowser/view/DEMO_GIND__custom_10293339/bookmark/table?lang=en&bookmarkId=cdf29d2c-8d15-4f2c-96b6-a51f8a389103&c=1709906392638

eu_owners = pd.read_excel("eu_owners.xlsx")
eu_hh_size = pd.read_excel("eu_hh_size.xlsx")
eu_demo_size = pd.read_excel("eu_demo_size.xlsx")

eu_owners = eu_owners.iloc[11:38,0:2]
eu_owners = eu_owners.reset_index(drop=True)
eu_owners.columns = ["Country", "Owner share"]
eu_owners = eu_owners.set_index('Country').loc[eu27_list].reset_index()

eu_hh_size = eu_hh_size.iloc[8:35,0:2]
eu_hh_size = eu_hh_size.reset_index(drop=True)
eu_hh_size.columns = ["Country", "Avg household size"]
eu_hh_size = eu_hh_size.set_index('Country').loc[eu27_list].reset_index()

eu_demo_size = eu_demo_size.iloc[8:35,0:2]
eu_demo_size = eu_demo_size.reset_index(drop=True)
eu_demo_size.columns = ["Country", "Population"]
eu_demo_size = eu_demo_size.set_index('Country').loc[eu27_list].reset_index()


eu_owners["Avg household size"] = eu_hh_size["Avg household size"]
eu_owners["Population"] = eu_demo_size["Population"]

eu_owners["Houses owned by households"] = (eu_owners["Population"] / eu_owners["Avg household size"]) * eu_owners["Owner share"] / 100
eu_owners["Houses owned by households"] = eu_owners["Houses owned by households"].round(0).astype(int)
eu_owners["Share of house ownership"] = eu_owners["Houses owned by households"] / eu_owners["Houses owned by households"].sum()

eu_owners = eu_owners.T 
eu_owners.columns = eu_owners.iloc[0]
eu_owners = eu_owners[1:]
#eu_owners.iloc[-1,:]
eu_owners

Country,Austria,Belgium,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,Finland,France,Germany,Greece,Hungary,Ireland,Italy,Latvia,Lithuania,Luxembourg,Malta,Netherlands,Poland,Portugal,Romania,Slovakia,Slovenia,Spain,Sweden
Owner share,54.3,71.9,86.1,91.2,70.1,76,60,80.7,69.2,63.1,47.6,69.6,90.5,69.4,75.2,82.8,88.8,67.6,74.7,69.3,87.3,76,95.6,93.6,75.2,75.3,64.9
Avg household size,2.2,2.3,2.3,2.7,2.5,2.3,2,2.1,1.9,2.2,2,2.4,2.3,2.7,2.2,2.2,1.9,2.3,2.4,2.1,2.9,2.5,2.5,3.1,2.4,2.5,2
Population,9104772,11742796,6447710,3850894,949084,10827529,5932654,1365884,5563970,68277210,83118501,10413982,9599744,5271395,58997201,1892103,2857279,660809,542051,17811291,36753736,10516621,19054548,5428792,2116972,48085361,10521556
Houses owned by households,2247223,3670900,2413686,1300746,266123,3577792,1779796,524889,2026456,19583145,19782203,3020054,3777290,1354943,20166315,712118,1335401,194220,168713,5877726,11064141,3197052,7286459,1639144,663317,14483310,3414244
Share of house ownership,0.016581,0.027086,0.01781,0.009598,0.001964,0.026399,0.013132,0.003873,0.014952,0.144496,0.145965,0.022284,0.027871,0.009998,0.148799,0.005254,0.009853,0.001433,0.001245,0.043369,0.081638,0.02359,0.053764,0.012095,0.004894,0.106866,0.025192


In [9]:
eu_owners.to_pickle("eu27_ownership.xp")